In [1]:
!rm -rf banking_simulation_normal.log
!rm -rf banking_simulation_abnormal.log

In [2]:
import requests
import json
import warnings
import mysql.connector
import string
import random
warnings.filterwarnings('ignore')
import time
import random
import logging
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from pprint import pprint
from faker import Faker
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
formatter = logging.Formatter('%(levelname)s: %(message)s')


logger_normal = logging.getLogger("banking-simulation-normal")
logger_normal.setLevel(logging.DEBUG)

fh_normal = logging.FileHandler("banking_simulation_normal.log")
fh_normal.setLevel(logging.DEBUG)
fh_normal.setFormatter(formatter)

logger_normal.addHandler(fh_normal)

In [4]:
logger_normal.handlers

[<FileHandler /Users/luisherrera/Documents/git-repositories/utec-tesis-ii/banking_simulation_normal.log (DEBUG)>]

In [5]:
# logger_normal.info("INFO")
# logger_normal.warning("WARNING")
# logger_normal.debug("DEBUG")
# logger_normal.error("ERROR")

In [6]:
logger_abnormal = logging.getLogger("banking-simulation-abnormal")
logger_abnormal.setLevel(logging.DEBUG)

fh_abnormal = logging.FileHandler("banking_simulation_abnormal.log")
fh_abnormal.setLevel(logging.DEBUG)
fh_abnormal.setFormatter(formatter)

logger_abnormal.addHandler(fh_abnormal)

In [7]:
logger_abnormal.handlers

[<FileHandler /Users/luisherrera/Documents/git-repositories/utec-tesis-ii/banking_simulation_abnormal.log (DEBUG)>]

In [8]:
# logger_abnormal.info("INFO")
# logger_abnormal.warning("WARNING")
# logger_abnormal.debug("DEBUG")
# logger_abnormal.error("ERROR")

In [9]:
fake = Faker()
num_usernames = 200
users = [fake.user_name() for _ in range(num_usernames)]

# users = ["alice", "bob", "charlie", "david", "eve", "frank", "grace", "hannah", "ivy", "jack",
#          "katherine", "liam", "mia", "noah", "olivia", "penelope", "quinn", "ryan", "sophia",
#          "thomas", "ursula", "victor", "willow", "xander", "yasmine", "zane", "nora", "oscar",
#          "lucy", "jacob", "emma", "ava", "lily", "mason", "ella", "oliver", "james", "harper",
#          "benjamin", "amelia", "sophie", "ethan", "aiden", "chloe", "zoe", "leah", "daniel"]

print(users[:20])

['kevin59', 'murraykatie', 'mhiggins', 'smithkenneth', 'vanessamason', 'ugreene', 'jenniferthomas', 'andersonlauren', 'clarkheather', 'juan69', 'hurstlinda', 'stacyjones', 'jwalker', 'qbooker', 'kennedyjared', 'ubrown', 'derrickgreen', 'stephencrane', 'jonesclinton', 'cynthiamcbride']


In [10]:
# Function to randomly select a user from the list
def get_random_user():
    return random.choice(users)


# Function to generate a random IP address
def generate_random_ip():
    return f"{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}"


# Function to generate random locations
def generate_random_location():
    locations = ["New York, USA", "Berlin, Germany", "Mumbai, India", "London, UK", "Tokyo, Japan"]
    return random.choice(locations)


# Function to generate random device ID
def generate_random_device_id():
    return ''.join(random.choice('0123456789ABCDEF') for _ in range(6))

In [11]:
def generate_noise_log(log_name, usage_percentage, user):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    
    prefix = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - "
    
    if log_name == "Memory usage":
        if usage_percentage > 95:
            log_type = "ERROR"
            log_message = f"{log_name}: {usage_percentage}% used. High memory usage!"
        else:
            log_type = "INFO" if usage_percentage < 80 else "WARNING" if usage_percentage < 90 else "ERROR"
            log_message = f"{log_name}: {usage_percentage}% used."
    elif log_name == "CPU utilization":
        if usage_percentage > 95:
            log_type = "ERROR"
            log_message = f"{log_name}: {usage_percentage}% utilized. High CPU usage!"
        else:
            log_type = "INFO" if usage_percentage < 80 else "WARNING" if usage_percentage < 90 else "ERROR"
            log_message = f"{log_name}: {usage_percentage}% utilized."
    elif log_name == "Server status":
        server_statuses = [
            "All systems are operational.",
            "Healthy and responsive.",
            "Stable and performing well.",
            "Minor issues detected, but operational.",
            "Service interruption in progress. Investigating...",
        ]
        log_message = f"{log_name}: {random.choice(server_statuses)}"
        log_type = "INFO"
    elif log_name == "Network activity":
        network_activity_logs = [
            "Abnormal network traffic detected.",
            "Network congestion warning.",
            "Elevated network traffic."
        ]
        log_message = f"{log_name}: {random.choice(network_activity_logs)}"
        log_type = "INFO"
    elif log_name == "Execution time":
        execution_time = random.uniform(1.0, 5.0)
        log_message = f"{log_name}: Execution time: {execution_time:.2f} seconds."
        log_type = "DEBUG"
    else:
        if usage_percentage < 5:
            log_type = "ERROR"
            log_message = f"{log_name}: Disk space critically low, only {usage_percentage}% free."
        else:
            log_type = "INFO" if usage_percentage < 80 else "WARNING" if usage_percentage < 90 else "ERROR"
            log_message = f"{log_name}: {usage_percentage}% free."
    
    return prefix + log_message, log_type

# Function to generate random noise logs with varying log types and percentages
def generate_noise_logs(user):
    noise_logs = []
    
#     # Server status log
#     server_status_log, server_status_log_type = generate_noise_log("Server status", 0, user)
#     noise_logs.append((server_status_log, server_status_log_type))
    
    # Memory log
    memory_usage = random.randint(0, 100)
    memory_log, memory_log_type = generate_noise_log("Memory usage", memory_usage, user)
    noise_logs.append((memory_log, memory_log_type))
    
    # CPU log
    cpu_usage = random.randint(0, 100)
    cpu_log, cpu_log_type = generate_noise_log("CPU utilization", cpu_usage, user)
    noise_logs.append((cpu_log, cpu_log_type))
    
#     # Network activity log (always WARNING)
#     server_status_log, server_status_log_type = generate_noise_log("Network activity", 0, user)
#     noise_logs.append((server_status_log, server_status_log_type))

    # Disk space log
    disk_space = random.randint(0, 100)
    disk_space_log, disk_space_log_type = generate_noise_log("Disk space", disk_space, user)
    noise_logs.append((disk_space_log, disk_space_log_type))

    # Execution time log (always DEBUG)
    execution_time_log, execution_time_log_type = generate_noise_log("Execution time", 0, user)
    noise_logs.append((execution_time_log, execution_time_log_type))
    
    return noise_logs

def enrich_logs(normal_logs, user):
    noise_logs = generate_noise_logs(user)
    combined_logs = []
    
    order = random.randint(0, 1)

    while normal_logs or noise_logs:
        if order == 0 and normal_logs:
            combined_logs.append((*normal_logs.pop(0), False))
        elif order == 1 and noise_logs:
            combined_logs.append((*noise_logs.pop(0), True))
        order = random.randint(0, 1)

    return combined_logs

## Authentication Scenarios

In [12]:
# Function to simulate successful login attempts
def simulate_successful_login():
    user = get_random_user()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - Successful login attempt from user {user}."
    return enrich_logs([(log_message, "INFO")], user)

    
# Function to simulate single failed login attempt followed by a successful login
def simulate_failed_then_successful_login():
    user = get_random_user()
    timestamp1 = time.strftime("%Y-%m-%d %H:%M:%S")
    timestamp2 = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time() + 5 * 60))  # 5 minutes later
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()

    # Failed login
    log_message1 = f"{timestamp1} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - Login attempt failed for user {user}."

    # Successful login
    log_message2 = f"{timestamp2} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {user} was able to login successfully after a previous failed attempt."
    return enrich_logs([(log_message1, "INFO"), (log_message2, "INFO")], user)


# Function to simulate multiple consecutive failed login attempts
def simulate_multiple_failed_attempts():
    user = get_random_user()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    messages = []
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - User {user} could not login into the system after a previous failed attempt."
    for _ in range(random.randint(3, 5)):  # Simulate 3 to 5 consecutive failed attempts
        messages.append((log_message, "INFO"))
    return enrich_logs(messages, user)

        
# # Function to simulate successful login attempts from multiple locations
# def simulate_successful_login_from_multiple_locations():
#     user = get_random_user()
#     messages = []
#     for _ in range(random.randint(3, 5)):
#         timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
#         ip = generate_random_ip()
#         location = generate_random_location()
#         device_id = generate_random_device_id()
#         log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - User was able to login into the system {user}."
#         messages.append((log_message, "INFO"))
#     return enrich_logs(messages, user)


# Function to simulate unsuccessful login attempts from geographically distant locations
def simulate_geographically_distant_attempts():
    user = get_random_user()
    messages = []
    for _ in range(random.randint(3, 5)):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        ip = generate_random_ip()
        location = generate_random_location()
        device_id = generate_random_device_id()
        log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - Consecutive failed login attempt from user {user} from a different location."
        messages.append((log_message, "INFO"))
    return enrich_logs(messages, user)

# Simulate normal authentication scenarios
pprint(simulate_successful_login())
print()
pprint(simulate_failed_then_successful_login())
print()
pprint(simulate_multiple_failed_attempts())
print()
# pprint(simulate_successful_login_from_multiple_locations())
# print()
pprint(simulate_geographically_distant_attempts())

[('2023-11-17 01:18:21 - IP: 178.34.143.134 - Location: Berlin, Germany - '
  'Device ID: CB09AD - User: matthewsjeffrey - Successful login attempt from '
  'user matthewsjeffrey.',
  'INFO',
  False),
 ('2023-11-17 01:18:21 - IP: 199.200.27.224 - Location: Mumbai, India - Device '
  'ID: 5C4169 - User: matthewsjeffrey - Memory usage: 43% used.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 115.181.223.111 - Location: London, UK - Device '
  'ID: FB089D - User: matthewsjeffrey - CPU utilization: 98% utilized. High '
  'CPU usage!',
  'ERROR',
  True),
 ('2023-11-17 01:18:21 - IP: 61.253.16.195 - Location: Tokyo, Japan - Device '
  'ID: 3C798B - User: matthewsjeffrey - Disk space: 33% free.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 198.15.52.143 - Location: Berlin, Germany - '
  'Device ID: 5931AF - User: matthewsjeffrey - Execution time: Execution time: '
  '1.60 seconds.',
  'DEBUG',
  True)]

[('2023-11-17 01:18:21 - IP: 67.243.151.184 - Location: Mumbai, India - Device '


In [13]:
# Function to generate a random user account
def generate_random_account():
    return f"User{random.randint(1, 1000)}"

# Function to generate a random transaction amount within the given range
def generate_random_amount(min_amount, max_amount):
    return random.randint(min_amount, max_amount)

# Function to generate a random 12-digit account number with masked last 4 digits
def generate_random_account_number():
    prefix = ''.join(random.choices(string.digits, k=8))
    last_4_digits = ''.join(random.choices(string.digits, k=4))
    masked_account_number = prefix + 'XXXX'  # Replace the first 8 digits with 'X'
    return masked_account_number

# Function to generate a random 4-digit transaction code
def generate_random_transaction_code():
    return ''.join(random.choices(string.digits, k=4))

# Function to generate a generic operation sentence
def generate_operation_sentence(user, operation_type, amount, success=True, reason=None):
    operation_result = "Successful" if success else "Failed"
    operation_reason = f"Reason for failure: {reason}" if not success and reason else ""
    return f"{operation_result} {operation_type} of {amount} units. {operation_reason}"
 

# Function to simulate regular and consistent deposit amounts and withdrawal amounts
def simulate_regular_transaction(transaction_type, user=None, timestamp=None, ip=None, location=None, device_id=None):
    user = get_random_user() if user is None else user
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S") if timestamp is None else timestamp
    amount = generate_random_amount(100, 1000) if transaction_type == "deposit" else generate_random_amount(50, 500)
#     operation_sentence = generate_operation_sentence(user, transaction_type, amount)
    operation_sentence = f"Regular {transaction_type} of {amount} units."
    ip = generate_random_ip() if ip is None else ip
    location = generate_random_location() if location is None else location
    device_id = generate_random_device_id() if device_id is None else device_id
    
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)


def simulate_regular_deposit():
    return simulate_regular_transaction("deposit")

    
def simulate_regular_withdrawal():
    return simulate_regular_transaction("withdrawal")


# Function to simulate deposits and withdrawals within available balance
def simulate_within_balance_transaction(transaction_type):
    user = get_random_user()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    balance = generate_random_amount(500, 5000)
    amount = generate_random_amount(100, 1000) if transaction_type == "deposit" else generate_random_amount(50, 500)
    # operation_sentence = generate_operation_sentence(user, transaction_type, amount)
    operation_sentence = f"Normal {transaction_type} of {amount} units within allowed limits."
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)


# Function to simulate consistent transaction frequencies
def simulate_consistent_frequencies(transaction_type):
    user = get_random_user()
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    current_date = datetime.now()
    messages = []
    for _ in range(random.randint(1, 3)):  # Simulate 1 to 3 transactions per day
        days_apart = random.randint(1, 3)
        timestamp = current_date + timedelta(days=days_apart)
        current_date = timestamp
        timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")
        message = simulate_regular_transaction(transaction_type, user, timestamp, ip, location, device_id)
        messages.extend(message)
    return messages


def simulate_consistent_deposits():
    return simulate_consistent_frequencies("deposit")


def simulate_consistent_withdrawals():
    return simulate_consistent_frequencies("withdrawal")

        
# Function to simulate large withdrawals exceeding a predefined threshold
def simulate_large_withdrawal():
    user = get_random_user()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    amount = generate_random_amount(10001, 20000)
    # operation_sentence = generate_operation_sentence(user, "withdrawal", amount)
    operation_sentence = f"Large withdrawal of {amount} units."
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)

# Function to simulate multiple rapid withdrawals within a 1-hour window
def simulate_multiple_rapid_withdrawals():
    user = get_random_user()
    ip = generate_random_ip()  # Use a single IP for all withdrawals
    location = generate_random_location()  # Use a single location for all withdrawals
    device_id = generate_random_device_id()  # Use a single device ID for all withdrawals
    rapid_count = random.randint(5, 10)  # Randomize the number of rapid withdrawals

    current_date = datetime.now()
    messages = []
    for i in range(rapid_count):
        timestamp = current_date + timedelta(minutes=random.randint(1, 3))
        current_date = timestamp
        timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")
        amount = generate_random_amount(50, 500)
        operation_sentence = f"Rapid withdrawal of {amount} units"
        log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
        messages.append((log_message, "INFO"))

    return enrich_logs(messages, user)


# Function to simulate deposits and withdrawals from unusual or unauthorized user accounts
def simulate_unusual_accounts_transaction(transaction_type):
    user = get_random_user()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    unusual_user = generate_random_account()
    amount = generate_random_amount(100, 1000) if transaction_type == "deposit" else generate_random_amount(50, 500)
    # operation_sentence = generate_operation_sentence(user, transaction_type, amount, False, "Unusual account")
    operation_sentence = f"Irregular {transaction_type} of {amount} units from unusual account."
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}."
    return enrich_logs([(log_message, "INFO")], user)

 
def simulate_unusual_accounts_deposits():
    return simulate_unusual_accounts_transaction("deposit")

    
def simulate_unusual_accounts_withdrawals():
    return simulate_unusual_accounts_transaction("withdrawal")


# Function to simulate multiple failed withdrawal attempts due to insufficient funds
def simulate_multiple_failed_withdrawals():
    user = get_random_user()
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    
    # Simulate 3 consecutive failed withdrawal attempts
    messages = []
    current_date = datetime.now()
    for i in range(3):
        timestamp = current_date + timedelta(minutes=random.randint(1, 3))
        current_date = timestamp
        timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")
        amount = generate_random_amount(50, 500)
        account_number = generate_random_account_number()
        transaction_code = generate_random_transaction_code()
        
        #operation_sentence = generate_operation_sentence(user, "withdrawal", amount, False, "Insufficient funds")
        operation_sentence = f"Failed withdrawal of {amount} units due to insufficient funds."
        
        log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
        messages.append((log_message, "INFO"))

    return enrich_logs(messages, user)

# Function to simulate withdrawals from dormant or rarely used accounts
def simulate_dormant_account_withdrawal():
    user = get_random_user()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    dormant_user = generate_random_account()
    amount = generate_random_amount(100, 1000)
    # operation_sentence = generate_operation_sentence(user, "withdrawal", amount, False, "Dormant account")
    operation_sentence = f"Unsual withdrawal from dormant account for {amount} units"
    ip = generate_random_ip()
    location = generate_random_location()
    device_id = generate_random_device_id()
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)

# Simulate normal debit transaction scenarios
simulate_regular_deposit() + simulate_regular_withdrawal() + simulate_consistent_deposits() + simulate_consistent_withdrawals()

# Simulate abnormal debit transaction scenarios
simulate_large_withdrawal() + simulate_multiple_rapid_withdrawals() + simulate_unusual_accounts_deposits() + simulate_unusual_accounts_withdrawals() + simulate_multiple_failed_withdrawals() + simulate_dormant_account_withdrawal()

[('2023-11-17 01:18:21 - IP: 156.252.223.157 - Location: New York, USA - Device ID: 96674A - User: brianburke - Memory usage: 52% used.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 165.47.132.120 - Location: Tokyo, Japan - Device ID: B930FA - User: brianburke - CPU utilization: 91% utilized.',
  'ERROR',
  True),
 ('2023-11-17 01:18:21 - IP: 225.188.225.207 - Location: London, UK - Device ID: 268ADA - User: brianburke - Large withdrawal of 12237 units.',
  'INFO',
  False),
 ('2023-11-17 01:18:21 - IP: 25.162.64.144 - Location: Tokyo, Japan - Device ID: B23CD0 - User: brianburke - Disk space: 64% free.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 139.111.141.51 - Location: Tokyo, Japan - Device ID: 477427 - User: brianburke - Execution time: Execution time: 1.07 seconds.',
  'DEBUG',
  True),
 ('2023-11-17 01:20:21 - IP: 82.115.206.220 - Location: Mumbai, India - Device ID: 47BD85 - User: amber97 - Rapid withdrawal of 125 units',
  'INFO',
  False),
 ('2023-11-17 01:23:21 - I

In [14]:
def generate_cc_operation_sentence(user, amount, success=True, reason=None):
    operation_result = "Successful credit card transaction" if success else "Failed credit card transaction"
    operation_reason = f"Reason for failure: {reason}" if not success and reason else ""
    return f"{operation_result} of {amount} units. {operation_reason}"

# Normal Behavior 1: Credit card transactions within the credit limit of the cardholder
def simulate_within_credit_limit():
    user = get_random_user()
    location = generate_random_location()
    device_id = generate_random_device_id()
    ip = generate_random_ip()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    amount = random.randint(50, 500)
    # operation_sentence = generate_cc_operation_sentence(user, amount, True)
    operation_sentence = "Successful credit card transaction within credit limits of {amount} units."
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)


# Abnormal Behavior 1: Large, unexpected transactions
def simulate_large_transaction():
    user = get_random_user()
    location = generate_random_location()
    device_id = generate_random_device_id()
    ip = generate_random_ip()
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    amount = random.randint(10000, 50000)
    #operation_sentence = generate_cc_operation_sentence(user, amount, False, "Large credit card transaction that significantly exceeds the card's credit limit.")
    operation_sentence = f"Large credit card transaction of {amount} units that significantly exceeds the card's credit limit."
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)


# Abnormal Behavior 2: Rapid and consecutive transactions
def simulate_rapid_transactions():
    user = get_random_user()
    location = generate_random_location()
    device_id = generate_random_device_id()
    ip = generate_random_ip()
    rapid_count = random.randint(5, 10)  # Randomize the number of rapid withdrawals

    messages = []
    current_date = datetime.now()
    for i in range(rapid_count):
        amount = generate_random_amount(50, 500)
        timestamp = current_date + timedelta(minutes=random.randint(1, 3))
        current_date = timestamp
        timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")
        operation_sentence = f"Irregular rapid credit card transaction of {amount} units."
        log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
        messages.append((log_message, "INFO"))


    return enrich_logs(messages, user)


# Abnormal Behavior 3: Transactions that involve high-risk merchant categories
def simulate_high_risk_transactions():
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    user = get_random_user()
    location = generate_random_location()
    device_id = generate_random_device_id()
    ip = generate_random_ip()
    amount = random.randint(50, 500)
    # operation_sentence = generate_cc_operation_sentence(user, amount, False, "Credit card transaction involving a high-risk merchant category or known fraudulent business.")
    operation_sentence = f"Credit card transaction involving a high-risk merchant category or known fraudulent business for {amount} units."
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)


# Abnormal Behavior 5: Unusual transaction times or dates
def simulate_unusual_transaction_times():
    user = get_random_user()
    location = generate_random_location()
    device_id = generate_random_device_id()
    ip = generate_random_ip()
    start_time = datetime.strptime("00:00:00", "%H:%M:%S").time()
    end_time = datetime.strptime("06:00:00", "%H:%M:%S").time()
    timestamp = datetime.combine(datetime.today(), start_time) + timedelta(seconds=random.randint(0, (end_time.hour * 3600 + end_time.minute * 60 + end_time.second)))
    timestamp = timestamp.strftime("%Y-%m-%d %H:%M:%S")
    amount = random.randint(50, 500)
    # operation_sentence = generate_cc_operation_sentence(user, amount, False, "Late-night credit card transaction that is an out-of-pattern purchases.")
    operation_sentence = f"Late-night credit card transaction of {amount} units that is an out-of-pattern purchases."
    log_message = f"{timestamp} - IP: {ip} - Location: {location} - Device ID: {device_id} - User: {user} - {operation_sentence}"
    return enrich_logs([(log_message, "INFO")], user)


# Simulate credit card transaction scenarios
simulate_within_credit_limit() + simulate_large_transaction() + simulate_rapid_transactions() + simulate_high_risk_transactions() + simulate_unusual_transaction_times()

[('2023-11-17 01:18:21 - IP: 245.149.86.210 - Location: New York, USA - Device ID: E92CC5 - User: singherika - Memory usage: 10% used.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 143.141.177.107 - Location: New York, USA - Device ID: 278220 - User: singherika - Successful credit card transaction within credit limits of {amount} units.',
  'INFO',
  False),
 ('2023-11-17 01:18:21 - IP: 181.98.158.44 - Location: Mumbai, India - Device ID: 60302A - User: singherika - CPU utilization: 63% utilized.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 209.125.195.28 - Location: New York, USA - Device ID: 655EE0 - User: singherika - Disk space: 10% free.',
  'INFO',
  True),
 ('2023-11-17 01:18:21 - IP: 70.99.75.65 - Location: Mumbai, India - Device ID: ABF2C3 - User: singherika - Execution time: Execution time: 3.82 seconds.',
  'DEBUG',
  True),
 ("2023-11-17 01:18:21 - IP: 147.61.236.64 - Location: London, UK - Device ID: 0A0D3C - User: mgonzalez - Large credit card transaction of 4310

In [15]:
import random
from datetime import datetime, timedelta

class SyntheticDatasetGenerator:
    def __init__(self, size=10000, normal_percentage=0.95, abnormal_percentage=0.05):
        self.size = size
        self.normal_percentage = normal_percentage
        self.abnormal_percentage = abnormal_percentage
        self.authentication_functions = [
            (simulate_successful_login, 0),
            (simulate_failed_then_successful_login, 0),
            (simulate_multiple_failed_attempts, 1),
            (simulate_geographically_distant_attempts, 1),
        ]
        self.transaction_functions = [
            (simulate_regular_deposit, 0),
            (simulate_regular_withdrawal, 0),
            (simulate_consistent_deposits, 0),
            (simulate_consistent_withdrawals, 0),
            (simulate_large_withdrawal, 1),
            (simulate_unusual_accounts_deposits, 1),
            (simulate_unusual_accounts_withdrawals, 1),
            (simulate_multiple_failed_withdrawals, 1),
            (simulate_dormant_account_withdrawal, 1)
        ]
        self.credit_card_functions = [
            (simulate_within_credit_limit, 0),
            (simulate_large_transaction, 1),
            (simulate_rapid_transactions, 1),
            (simulate_high_risk_transactions, 1),
            (simulate_unusual_transaction_times, 1)
        ]


    def generate_logs(self, function_tuple_list):
        logs = []
        labels = []
        function_names = []
        
        normal_function_tuples = [(func, log_type) for func, log_type in function_tuple_list if log_type == 0]
        abnormal_function_tuples = [(func, log_type) for func, log_type in function_tuple_list if log_type == 1]

        while len(logs) < self.size:
            # Randomly select a function based on the specified percentages
            if random.random() < self.normal_percentage:
                function, log_type = random.choice(normal_function_tuples)
            else:
                function, log_type = random.choice(abnormal_function_tuples)

            # Generate logs using the selected function
            result_logs = function()
            logs.extend(result_logs)
            function_names.extend([function.__name__] * len(result_logs))

            # Append the log to the appropriate list
            if log_type == 0:
                labels.extend([0] * len(result_logs))
            else:
                labels.extend([1] * len(result_logs))

        return logs, labels, function_names

    def generate_authentication_logs(self):
        return self.generate_logs(self.authentication_functions)

    def generate_transaction_logs(self):
        return self.generate_logs(self.transaction_functions)

    def generate_credit_card_logs(self):
        return self.generate_logs(self.credit_card_functions)

    def generate_synthetic_dataset(self):
        # Generate normal and abnormal logs for each category
        authentication_logs, authentication_labels, authentication_func_names = self.generate_authentication_logs()
        transaction_logs,  transaction_labels, transaction_func_names = self.generate_transaction_logs()
        credit_card_logs, credit_card_labels, credit_card_func_names = self.generate_credit_card_logs()

        # Combine logs from different categories
        combined_logs = authentication_logs + transaction_logs + credit_card_logs
        combined_labels = authentication_labels + transaction_labels + credit_card_labels
        combined_func_names = authentication_func_names + transaction_func_names + credit_card_func_names

        return combined_logs, combined_labels, combined_func_names

In [16]:
%%time
logs_tuples, labels, function_names = SyntheticDatasetGenerator(size=30000).generate_synthetic_dataset()
logs, log_type, is_noise = zip(*logs_tuples)
logs = list(logs)
log_type = list(log_type)
is_noise = list(is_noise)

CPU times: user 1 s, sys: 21.5 ms, total: 1.02 s
Wall time: 1.03 s


In [17]:
len(logs), len(log_type), len(labels), len(function_names)

(90004, 90004, 90004, 90004)

In [18]:
df = pd.DataFrame({"logs": logs, "log_type": log_type, "noise": is_noise, "labels": labels, "names": function_names})
df.head()

,logs,log_type,noise,labels,names
0,2023-11-17 01:18:21 - IP: 100.250.36.65 - Loca...,INFO,True,0,simulate_failed_then_successful_login
1,2023-11-17 01:18:21 - IP: 32.25.189.234 - Loca...,INFO,False,0,simulate_failed_then_successful_login
2,2023-11-17 01:23:21 - IP: 32.25.189.234 - Loca...,INFO,False,0,simulate_failed_then_successful_login
3,2023-11-17 01:18:21 - IP: 130.21.128.181 - Loc...,INFO,True,0,simulate_failed_then_successful_login
4,2023-11-17 01:18:21 - IP: 172.210.38.44 - Loca...,INFO,True,0,simulate_failed_then_successful_login


In [19]:
df.labels.value_counts()

0    84728
1     5276
Name: labels, dtype: int64

In [20]:
df.names.value_counts()

simulate_within_credit_limit                27995
simulate_failed_then_successful_login       15168
simulate_successful_login                   12675
simulate_consistent_deposits                 9485
simulate_consistent_withdrawals              9465
simulate_regular_deposit                     5165
simulate_regular_withdrawal                  4775
simulate_multiple_failed_attempts            1161
simulate_geographically_distant_attempts      996
simulate_rapid_transactions                   811
simulate_high_risk_transactions               435
simulate_unusual_transaction_times            395
simulate_large_transaction                    365
simulate_multiple_failed_withdrawals          273
simulate_large_withdrawal                     230
simulate_unusual_accounts_withdrawals         225
simulate_dormant_account_withdrawal           195
simulate_unusual_accounts_deposits            190
Name: names, dtype: int64

In [21]:
df_wo_noise = df[df.noise == False]
df_wo_noise.shape

(21092, 5)

In [22]:
df_wo_noise.labels.value_counts()

0    18968
1     2124
Name: labels, dtype: int64

In [23]:
df_wo_noise.names.value_counts()

simulate_within_credit_limit                5599
simulate_failed_then_successful_login       5056
simulate_successful_login                   2535
simulate_consistent_deposits                1897
simulate_consistent_withdrawals             1893
simulate_regular_deposit                    1033
simulate_regular_withdrawal                  955
simulate_multiple_failed_attempts            573
simulate_rapid_transactions                  527
simulate_geographically_distant_attempts     500
simulate_multiple_failed_withdrawals         117
simulate_high_risk_transactions               87
simulate_unusual_transaction_times            79
simulate_large_transaction                    73
simulate_large_withdrawal                     46
simulate_unusual_accounts_withdrawals         45
simulate_dormant_account_withdrawal           39
simulate_unusual_accounts_deposits            38
Name: names, dtype: int64

In [24]:
normal_logs = df[df.labels == 0]
anormal_logs = df[df.labels == 1]

In [25]:
def write_logs(logger, df):
    for log, log_type in zip(df.logs.tolist(), df.log_type.tolist()):
        if log_type == "INFO":
            logger.info(log)
        elif log_type == "WARNING":
            logger.warning(log)
        elif log_type == "DEBUG":
            logger.debug(log)
        elif log_type == "ERROR":
            logger.error(log)
        else:
            print(f"ERROR: Invalid Log Type: {log_type}.")

In [26]:
write_logs(logger_normal, normal_logs)

In [27]:
write_logs(logger_abnormal, anormal_logs)

In [28]:
df_noise = df[df.noise == True]

In [29]:
df.shape

(90004, 5)

In [30]:
df_noise = df[df.noise == True]

In [31]:
df_noise.shape

(68912, 5)

In [32]:
for l in df_noise.logs.tolist()[:10]:
    print(l)

2023-11-17 01:18:21 - IP: 100.250.36.65 - Location: Mumbai, India - Device ID: 3A150E - User: kennedyjared - Memory usage: 34% used.
2023-11-17 01:18:21 - IP: 130.21.128.181 - Location: New York, USA - Device ID: E282DB - User: kennedyjared - CPU utilization: 63% utilized.
2023-11-17 01:18:21 - IP: 172.210.38.44 - Location: Berlin, Germany - Device ID: 368508 - User: kennedyjared - Disk space: 11% free.
2023-11-17 01:18:21 - IP: 90.54.170.153 - Location: London, UK - Device ID: 8E7BDF - User: kennedyjared - Execution time: Execution time: 2.21 seconds.
2023-11-17 01:18:21 - IP: 131.243.206.63 - Location: Tokyo, Japan - Device ID: 50AE6D - User: thomas40 - Memory usage: 76% used.
2023-11-17 01:18:21 - IP: 216.141.187.8 - Location: Tokyo, Japan - Device ID: 708D3E - User: thomas40 - CPU utilization: 46% utilized.
2023-11-17 01:18:21 - IP: 35.193.170.139 - Location: Mumbai, India - Device ID: 76DBE7 - User: thomas40 - Disk space: 52% free.
2023-11-17 01:18:21 - IP: 51.251.205.252 - Locati

In [33]:
!cp banking_simulation_normal.log data/banking_simulation
!cp banking_simulation_abnormal.log data/banking_simulation